# Generating a list of chemical names and CAS Registry Numbers

Function:  This notebook generates a database table of Chemical Abstract Service Registry Numbers from substance names. 
Rationale:  The database table allows the CAS Registry Number to be used as an index for subsequent manipulations with 
solubility parameter data, enabling rapid and unambiguous operations on solvent sets.  Using a large data set now will make the system extensible for widespread use later. Having a custom notebook do this job ensures compatibility and
isolates the solubility parameter code from dependency on external sources.

### Dependencies / Requirements

### Libraries
* xml (.etree.ElementTree)
* sqlalchemy 
* numpy
* pandas

### Files
* Folders `data_Sources` and `validation` co-located with the notebook.
* Within `data_Sources`, a sub-folder named `chemidplus`
* Within `chemidplus` an input file named `chemid_latest.xml`
* Within `validation` an input file named `hsp_validation.csv`

Special note:  due to the large size of the unzipped .xml file, it is not provided in the GitHub source.  Instead,
a zipped version is provided on GitHub.  When downloading source code from GitHub, unzip the file and place it in a folder
named chemidplus in Data_Sources.

* Note:  **Input file size is very large -- ~ 1 GB**

In [1]:
import datetime
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import sqlalchemy 
# Input file
infile = 'data_sources/chemidplus/chemid_latest.xml'
validation_file = 'validation/hsp_validation.csv'

### Import and Set-Up

In [2]:
# Fetch the input file and convert it to an XML tree -- root.tag should return
# 'file' if this executes correctly -- this step takes some time to execute
with open(infile,'r') as f:
    tree = ET.parse(infile)
root = tree.getroot()
root.tag

'file'

In [3]:
# FYI, print the number of chemicals in the root -- this should be several hundred k
len(root)

420658

In [4]:
# To read in the data, we will first make lists
# Later, we'll put these in DataFrames to process quickly
# For now, just make lists of name, ChemIDPlus-id, CASRN and CASRN, synonym
prim_name = []
chemid = []
casrn = []
casrn_syn = []
syn = []

for substance in range(len(root)):
    subst_name = root[substance][0].find('NameOfSubstance')
    subst_cas = root[substance][1].find('CASRegistryNumber')
    subst_id = root[substance].attrib['id']
    
    # Skip all list additions if there is not both a main name and CASRN
    if subst_name is not None and subst_cas is not None:
        prim_name.append(subst_name.text)
        chemid.append(subst_id)
        casrn.append(subst_cas.text)
        
        # Check if there is a systematic name
        subst_syst = root[substance][0].find('SystematicName')
        
        # Add it to synonym list if there is
        if subst_syst is not None:
            casrn_syn.append(subst_cas.text)
            syn.append(subst_syst.text)
            
        # Check if there are synonyms
        subst_syn = root[substance][0].findall('Synonyms')
        
        # Add if there are any ...
        if len(subst_syn) != 0:
            for synonym in subst_syn:
                casrn_syn.append(subst_cas.text)
                syn.append(synonym.text)
        

In [5]:
# Now, convert lists to DataFrames for further processing
name_CAS_df = pd.DataFrame({'Subst_Name':prim_name,'CASRN':casrn, 'id':chemid})
synonyms_df = pd.DataFrame({'CASRN':casrn_syn,'Synonym':syn})

In [6]:
# Get some DataFrame info on length and unique values
print (f'name_CAS has {len(name_CAS_df)} rows with {name_CAS_df.Subst_Name.nunique()} unique names.')
print (f'name_CAS has {name_CAS_df.CASRN.nunique()} unique CAS Registry Numbers.')
print (f'Synonyms table has {len(synonyms_df)} rows with {synonyms_df.Synonym.nunique()} synonyms.')

name_CAS has 120692 rows with 119838 unique names.
name_CAS has 120692 unique CAS Registry Numbers.
Synonyms table has 706245 rows with 690191 synonyms.


### Data Processing

What you should see is no duplicate CAS Registry Numbers, some duplicated names, and quite a few duplicated synonyms...
The following cells take care of the easy stuff ... 

In [7]:
# Drop cases where name, CAS pairs are duplicates -- there should not be any to start with
# but just in case there are, ths is a quick way that would fix it
name_CAS_df.drop_duplicates(inplace = True)
# For the synonyms column, drop duplicated CASRN, synonym pairs -- this usually represents a majority of cases
synonyms_df.drop_duplicates(inplace = True)

In [8]:
# Drop any rows of the Synonyms where the Synonym, CASRN pair is the same as a name, CASRN pair
synonyms_df = synonyms_df.merge(name_CAS_df, left_on = ['Synonym','CASRN'], right_on = ['Subst_Name','CASRN'], 
                                how='left', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
# This operation produces an extra column in synonyms_df that needs deletion
synonyms_df = synonyms_df.drop(columns = 'Subst_Name')

In [9]:
# Now check the results ...
print (f'name_CAS has {len(name_CAS_df)} rows with {name_CAS_df.Subst_Name.nunique()} unique names.')
print (f'name_CAS has {name_CAS_df.CASRN.nunique()} unique CAS Registry Numbers.')
print (f'Synonyms table has {len(synonyms_df)} rows with {synonyms_df.Synonym.nunique()} synonyms.')

name_CAS has 120692 rows with 119838 unique names.
name_CAS has 120692 unique CAS Registry Numbers.
Synonyms table has 643629 rows with 638199 synonyms.


To tackle ambiguous entries, we will make use of the fact that CAS Registry Numbers reflect the order of entry,
thus the more "common" compound is entered first.  We will assign a CAS priority number, sort on this, and 
only keep the duplicate with the lower number in the primary name table.  We will take the alternates to a separate
table for later use.

In [10]:
# Parse the CAS Registry Number -- the last digit is a checksum and can be dropped
name_CAS_df['CASRank'] = name_CAS_df.CASRN.str.split('-',3).apply(lambda x : x[0]).astype('int') * 100 \
                      + name_CAS_df.CASRN.str.split('-',3).apply(lambda x : x[1]).astype('int')
# Sort by this rank -- the source data is typically sorted already
name_CAS_df = name_CAS_df.sort_values(by='CASRank')
# Now create duplicate flags
name_CAS_df['Dup_Name_Flag'] = name_CAS_df.duplicated('Subst_Name',keep = False)
# Drop the rank columns as is it no longer needed
name_CAS_df = name_CAS_df.drop(columns='CASRank')

In [11]:
# First grab the duplicates 
alt_primary_name_df = name_CAS_df[name_CAS_df.duplicated('Subst_Name')]
# Now they can be safely dropped from the main table
name_CAS_df = name_CAS_df.drop_duplicates('Subst_Name')

In [12]:
# Now check the results ...
print (f'name_CAS has {len(name_CAS_df)} rows with {name_CAS_df.Subst_Name.nunique()} unique names.')
print (f'name_CAS has {name_CAS_df.CASRN.nunique()} unique CAS Registry Numbers.')
print (f'Synonyms table has {len(synonyms_df)} rows with {synonyms_df.Synonym.nunique()} synonyms.')

name_CAS has 119838 rows with 119838 unique names.
name_CAS has 119838 unique CAS Registry Numbers.
Synonyms table has 643629 rows with 638199 synonyms.


In [13]:
# Do for the synonyms table what was done for the name_CASRN table
# Parse the CAS Registry Number 
synonyms_df['CASRank'] = synonyms_df.CASRN.str.split('-',3).apply(lambda x : x[0]).astype('int') * 100 \
                      + synonyms_df.CASRN.str.split('-',3).apply(lambda x : x[1]).astype('int')
# Sort by this rank -- the source data is typically sorted already
synonyms_df = synonyms_df.sort_values(by='CASRank')
# Now create duplicate flags
synonyms_df['Dup_Name_Flag'] = synonyms_df.duplicated('Synonym',keep = False)
# Drop the rank columns as is it no longer needed
synonyms_df = synonyms_df.drop(columns='CASRank')

# First grab the duplicates 
alt_synonyms_df = synonyms_df[synonyms_df.duplicated('Synonym')]
# Now they can be safely dropped from the main table
synonyms_df = synonyms_df.drop_duplicates('Synonym')

# Now check the results ...
print (f'name_CAS has {len(name_CAS_df)} rows with {name_CAS_df.Subst_Name.nunique()} unique names.')
print (f'name_CAS has {name_CAS_df.CASRN.nunique()} unique CAS Registry Numbers.')
print (f'Synonyms table has {len(synonyms_df)} rows with {synonyms_df.Synonym.nunique()} synonyms.')

name_CAS has 119838 rows with 119838 unique names.
name_CAS has 119838 unique CAS Registry Numbers.
Synonyms table has 638199 rows with 638199 synonyms.


In [14]:
# Lastly, we are going to hand-craft some entries for the sake of useability.
# 1) "NMP" refers to the less common pyrrolidine rather than the more common pyrrolidinone, this is an accident
# of history due to the relatively recent widespread use of NMP.  This is common enough that it will certainly
# impact useability, so it will be corrected via hard-coding.  
synonyms_df.loc[synonyms_df.Synonym == 'NMP','CASRN'] = '872-50-4'
alt_synonyms_df.loc[alt_synonyms_df.Synonym == 'NMP','CASRN'] = '120-94-5'
# Other cases where common abbreviations reference correctly can be handled by the 'extra_synonyms' table.

### Data Validation
For validation, the CAS Registry Numbers are checked against names and synonyms from a known solvent set.  The default set
is a list of 145 of the most commonly used solvents  for Hansen Solubility Parameter determination, based on Hansen's key experimental work.  Custom validations can be added by placing the name and CAS Registry Number into the `hsp_validation.csv` file.

These hand-curated validations come from Sigma-Aldrich catalogs, Wikipedia, PubMed, and other on-line sources.  In all cases,
at least two sources were checked to validate the CAS Registry Number in the table.

The substances included in this list are those found in bold in Table A1 of Source 1 (Hansen's 2007 book), excluding salts, carbon dioxide, carbon disulfide, carbon tetrachloride, urea, and water (these have disputed HSP values), as well as complex mixtures such as Dupont dibasic esters, and a few compounds (listed under notes) that do not appear in the National Library of Medicine database.

In [17]:
# Read file into DataFrame
validate_df = pd.read_csv(validation_file)
length_check = len(validate_df)
validate_df.head()

,subst_name,synonym,casrn,delta-d,delta-p,delta-h,mol-vol,source_id,source_ref
0,Acetic acid,Ethanoic acid,64-19-7,14.5,8.0,13.5,57.1,1,A1.5
1,Acetic anhydride,Acetyl acetate,108-24-7,16.0,11.7,10.2,94.5,1,A1.6
2,Acetone,Dimethyl ketone,67-64-1,15.5,10.4,7.0,74.0,1,A1.7
3,Acetonitrile,Methyl cyanide,75-05-8,15.3,18.0,6.1,52.6,1,A1.10
4,Acetophenone,Acetylbenzene,98-86-2,19.6,8.6,3.7,117.4,1,A1.11


In [18]:
#Inner join on casrn should not produce new rows
validate_df = validate_df.merge(name_CAS_df,left_on='casrn',right_on='CASRN')
validate_df.head()

,subst_name,synonym,casrn,delta-d,delta-p,delta-h,mol-vol,source_id,source_ref,Subst_Name,CASRN,id,Dup_Name_Flag
0,Acetic acid,Ethanoic acid,64-19-7,14.5,8.0,13.5,57.1,1,A1.5,Acetic acid,64-19-7,0000064197,False
1,Acetic anhydride,Acetyl acetate,108-24-7,16.0,11.7,10.2,94.5,1,A1.6,Acetic anhydride,108-24-7,0000108247,False
2,Acetone,Dimethyl ketone,67-64-1,15.5,10.4,7.0,74.0,1,A1.7,Acetone,67-64-1,0000067641,False
3,Acetonitrile,Methyl cyanide,75-05-8,15.3,18.0,6.1,52.6,1,A1.10,Acetonitrile,75-05-8,0000075058,False
4,Acetophenone,Acetylbenzene,98-86-2,19.6,8.6,3.7,117.4,1,A1.11,Acetophenone,98-86-2,0000098862,False


In [19]:
# Test for any mismatch of substnce name
mismatch_df = validate_df.loc[validate_df['subst_name'] != validate_df['Subst_Name']]
len(mismatch_df)

0

In [20]:
# Now merge synonyms and test for difference in cas
validate_df = validate_df.merge(synonyms_df,left_on='synonym',right_on='Synonym')
validate_df.head()

,subst_name,synonym,casrn,delta-d,delta-p,delta-h,mol-vol,source_id,source_ref,Subst_Name,CASRN_x,id_x,Dup_Name_Flag_x,CASRN_y,Synonym,id_y,Dup_Name_Flag_y
0,Acetic acid,Ethanoic acid,64-19-7,14.5,8.0,13.5,57.1,1,A1.5,Acetic acid,64-19-7,0000064197,False,64-19-7,Ethanoic acid,NaN,False
1,Acetic anhydride,Acetyl acetate,108-24-7,16.0,11.7,10.2,94.5,1,A1.6,Acetic anhydride,108-24-7,0000108247,False,108-24-7,Acetyl acetate,NaN,False
2,Acetone,Dimethyl ketone,67-64-1,15.5,10.4,7.0,74.0,1,A1.7,Acetone,67-64-1,0000067641,False,67-64-1,Dimethyl ketone,NaN,False
3,Acetonitrile,Methyl cyanide,75-05-8,15.3,18.0,6.1,52.6,1,A1.10,Acetonitrile,75-05-8,0000075058,False,75-05-8,Methyl cyanide,NaN,False
4,Acetophenone,Acetylbenzene,98-86-2,19.6,8.6,3.7,117.4,1,A1.11,Acetophenone,98-86-2,0000098862,False,98-86-2,Acetylbenzene,NaN,False


In [21]:
mistmatch_df = validate_df.loc[validate_df['casrn'] != validate_df['CASRN_y']]
len(mismatch_df)

0

If the length of 'mismatch_df' is zero in both tests above, then for each validation case, the substance name, chosen synonym name, and CAS reference number all match, which means that in the vast majority of use cases the substance is guaranteed to be identified correctly.  The use of the National Library of Medicine provides a second layer of protection against erroneous data, as this database is actively maintained.

**Some special cases among the validations:**

In Hansen's book, "dioctyl adipate",  "dioctyl phthalate", and "trioctyl phosphate" are shown to refer to the n-octyl functional forms, but industrially, the structure is in fact the ethylhexyl form.  The NLM database assumes that "dioctyl" in these cases is a synonym for either.  The ambiguous name convention used for this database causes it to refer to the ehtyl hexyl form preferentially, which is almost certainly what users would want.  Given that Hansen's info for these compounds also states that they are commercial, I assume that the ethylhexyl forms were the ones measured.

"Dipropylene glycol" and "Dipropylene glycol methyl ether" are shown as a single chemical substance but labeled as commercial.  The commercial solvents are mixtures, and therefore the CAS Reference Number of the mixture is used in these cases.  The introduction to the tables in Hansen's book clearly states that items labeled as commercial may be mixtures, so this assumption is almost certainly valid, and it would be what users would expect.

"Ethyl lactate" is enantiomeric.  Hansen's table does not specify which form was used, but racemic is the most common and therefore was picked.  

Some of the glycol ethers in the Hansen book are not in the National Library of Medicine database, including ethylene glycol butyl ethyl ether, ethylene glycol methyl t-butyl ether, ethylene glycol di-t-butyl ether, ethylene glycol mono t-butyl ether, ethylene glycol monoisobutyl ether, and diethylene glycol methyl t-butyl ether.  Hexafluorohexanol is not in the NLM database and is thus excluded.

### Prepare Lists for SMILES Generation

Priority #1 will be the validation set.

In [23]:
# We want just a comma separated list for all the ChemIDPlus id's for use with PubChem
# The ChemIDPlus id has already been merged in as id_x, so we just write it to csv
validate_df.id_x.to_csv('validate_casrns.csv',index=False,header=False)

Next do some different-sized test runs.  Use CAS # for priority (lower will tend to be a more common compound).

In [25]:
name_1000_df = name_CAS_df.iloc[:1000,:]
name_1000_df.id.to_csv('first_1000_names.csv',index=False,header=False)

###  Notes

Source:  The National Library of Medicine provides the data source ChemIDPlus.  This source provides a monthly update
of basic chemical ID information.  It currently lists over 400,000 compounds, and is updated monthly.  Although the main use 
is for regulatory purposes, it is likely to provide an exhaustive list of solvents.  If making use of this data set, please
acknowledge the source and provide the link below to facilitate using the latest version of the data.  

The current version of the data is from 03-28-2019.  This script was last executed on:

In [20]:
print(datetime.datetime.now())

2019-07-05 19:49:58.002320


The URL for the latest ChemIDPlus download is:  ftp://ftp.nlm.nih.gov/nlmdata/.chemidlease/

CASRN or CAS Registry Number are registered trademarks of the chemical abstracts service.  Organizations that use this data
will need a license from the Chemical Abstracts Service.  The current notebook is for educational purposes only.

In [16]:
name_CAS_df.head()

,Subst_Name,CASRN,id,Dup_Name_Flag
0,Monobutyl phthalate,34-74-2,0000034742,True
1,Dimethyltubocurarine,35-67-6,0000035676,False
2,Carotenoids,36-88-4,0000036884,False
3,Holothurin A,38-26-6,0000038266,False
4,Methanal,50-00-0,0000050000,False


0000034742
